In [1]:
import re
import os
import sys
import math
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import lib.params as params
import lib.network as network

In [2]:
#read params

with open("config/train.params") as f:
    learning_rate=float(params.read_param(f.readline()))
    batch_size=int(params.read_param(f.readline()))
    epoch=int(params.read_param(f.readline()))
   
    print("training with a learning rate of {} for {} epochs with batchs of size {}".format(learning_rate,epoch,batch_size))

training with a learning rate of 0.01 for 1 epochs with batchs of size 10


In [3]:
data_all=np.load("out/all_data.npy")  
label_all=np.load("out/all_labels.npy")
print(data_all.shape,label_all.shape)

#prep for training
net=network.R2N2(learning_rate)
model_dir="model_{}_{}_{}".format(learning_rate,batch_size,epoch)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

(1, 24, 137, 137, 4) (1, 32, 32, 32)
encoder_network
(?, 24, 137, 137, 4)
(?, 24, 68, 68, 96)
(?, 24, 34, 34, 128)
(?, 24, 17, 17, 256)
(?, 24, 8, 8, 256)
(?, 24, 4, 4, 256)
(?, 24, 2, 2, 256)
(?, 24, 1024)
recurrent_module
(?, 4, 4, 4, 1, 256)
<unknown>
decoder_network


ValueError: as_list() is not defined on an unknown TensorShape.

In [ ]:
# train network
loss_all=[]
acc_all=[]
N=len(data_all)
for e in range(epoch):
    start_time=time.time()
    perm=np.random.permutation(N)
    data_all=data_all[perm]
    label_all=label_all[perm]
    split_size=math.ceil(N/batch_size)
    data_batchs=np.array_split(data_all,split_size)
    label_batchs=np.array_split(label_all,split_size)
    loss_epoch=[]
    acc_epoch=[]
    
    batch_number=0
    for data,label in zip(data_batchs,label_batchs):
        fd={net.X:data, net.Y: label}
        batch_info=net.sess.run([net.mean_loss,net.mean_accuracy,net.optimizing_op],feed_dict=fd)
        loss_batch,acc_batch=batch_info[0],batch_info[1]
        loss_epoch.append(loss_batch)
        acc_epoch.append(acc_batch)
        batch_number+=1
        if batch_number%50==0:
            print("epoch_{:03d}-batch_{:03d}: loss={}, acc={}".format(e,batch_number,loss_batch,acc_batch))
    loss_all.append(loss_epoch)
    acc_all.append(acc_epoch)
    net.save("{}/epoch_{:03d}".format(model_dir,e),loss_all,acc_all)
    print("epoch %d took %d seconds"%(e,time.time()-start_time))

In [ ]:
net_state=net.state(fd)

In [ ]:
net_state[0]

In [ ]:

utils.imshow_voxel(net.prediction.run